<a href="https://colab.research.google.com/github/LawTAGS/Deep-Learning-Classwork/blob/main/Assignments/Copy_of_10121091_Tugas_Classification_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. Binary Classification Using NumPy**

In [ ]:
import numpy as np

# Activation Functions
def softmax(z):
  exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
  return exp_z/np.sum(exp_z, axis=1, keepdims=True)

# Binary Cross-Entropy Loss
def binary_cross_entropy(y_true, y_pred):
  m = y_true.shape[0]
  loss = -np.mean(y_true * np.log(y_pred + 1e-9) + (1 - y_true) * np.log(1 - y_pred + 1e-9))
  return loss

# Multi-Layer Perceptron (MLP)
class BinaryMLP:
  def __init__(self, input_size, hidden_size):
    self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2 / input_size)
    self.b1 = np.zeros((1, hidden_size))
    self.W2 = np.random.randn(hidden_size, 2) * np.sqrt(2 / hidden_size)
    self.b2 = np.zeros((1, 2))

  def forward(self, X):
    self.z1 = np.dot(X, self.W1) + self.b1
    self.a1 = np.maximum(0, self.z1)
    self.z2 = np.dot(self.a1, self.W2) + self.b2
    self.a2 = softmax(self.z2)
    return self.a2

  def backward(self, X, y, learning_rate):
    m = X.shape[0]

    # Correct Softmax derivative for BCE: dL/dz = y_pred - y
    dz2 = self.a2 - y # Output Layer gradient (Softmax + BCE)
    dW2 = np.dot(self.a1.T, dz2) / m
    db2 = np.sum(dz2, axis=0, keepdims=True) / m

    # Hidden Layer error
    dz1 = np.dot(dz2, self.W2.T) * (self.z1 > 0)
    dW1 = np.dot(X.T, dz1) / m
    db1 = np.sum(dz1, axis=0, keepdims=True) / m

    # Update weights and biases
    self.W1 -= learning_rate * dW1
    self.b1 -= learning_rate * db1
    self.W2 -= learning_rate * dW2
    self.b2 -= learning_rate * db2

  def train(self, X, y, epochs=1000, learning_rate=0.01):
    for epoch in range(epochs):
      y_pred = self.forward(X)
      loss = binary_cross_entropy(y, y_pred)
      self.backward(X, y, learning_rate)

      if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

  def predict(self, X):
    y_pred = self.forward(X)
    return np.argmax(y_pred, axis=1)

# Generate Synthetic Binary Data
np.random.seed(10121091)

# Generate 2D features (simple binary classification)
X_data = np.random.randn(1000, 2)
y_data = (X_data[:, 0] + X_data[:, 1] > 0).astype(int)

# Convert labels to one-hot encoding for Softmax
y_data_one_hot = np.zeros((y_data.shape[0], 2))
y_data_one_hot[np.arange(y_data.size), y_data] = 1

# Split into training and testing sets (80% train, 20% test)
split_ratio = 0.7
split_index = int(len(X_data) * split_ratio)

X_train, X_test = X_data[:split_index], X_data[split_index:]
y_train, y_test = y_data_one_hot[:split_index], y_data_one_hot[split_index:]

# Train the MLP
mlp = BinaryMLP(input_size=2, hidden_size=10)
mlp.train(X_train, y_train, epochs=1000, learning_rate=0.01)

# Evaluate the Model
y_pred = mlp.predict(X_test)
y_true = np.argmax(y_test, axis=1)
accuracy = np.mean(y_pred == y_true)

print(f"Test Accuracy: {accuracy:.4f}%")

Epoch 0, Loss: 1.2962
Epoch 100, Loss: 0.4156
Epoch 200, Loss: 0.2773
Epoch 300, Loss: 0.2187
Epoch 400, Loss: 0.1828
Epoch 500, Loss: 0.1577
Epoch 600, Loss: 0.1392
Epoch 700, Loss: 0.1251
Epoch 800, Loss: 0.1139
Epoch 900, Loss: 0.1050
Test Accuracy: 0.9967%


## **2. Multi Layers Binary Classification Using NumPy**

In [ ]:
import numpy as np

# Activation Functions
def softmax(z):
    """Softmax activation function."""
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Prevents overflow
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# Binary Cross-Entropy Loss
def binary_cross_entropy(y_true, y_pred):
    """Computes Binary Cross-Entropy loss."""
    m = y_true.shape[0]
    loss = -np.mean(y_true * np.log(y_pred + 1e-9) + (1 - y_true) * np.log(1 - y_pred + 1e-9))  # Avoid log(0)
    return loss

# Multi-Layer Perceptron (MLP)
class CustomBinaryMLP:
    def __init__(self, layer_sizes):
        """
        Initializes a multi-layer MLP for binary classification.

        layer_sizes: List containing number of neurons in each layer, including input and output.
        Example: [2, 10, 5, 2] --> 2 inputs → 10 hidden → 5 hidden → 2 output (Softmax)
        """
        self.num_layers = len(layer_sizes) - 1
        self.weights = []
        self.biases = []

        # Initialize weights and biases
        for i in range(self.num_layers):
            self.weights.append(np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * np.sqrt(2 / layer_sizes[i]))  # He Init
            self.biases.append(np.zeros((1, layer_sizes[i + 1])))

    def forward(self, X):
        """
        Forward propagation through multiple layers.
        """
        self.activations = [X]
        self.z_values = []

        for i in range(self.num_layers - 1):  # Hidden layers
            z = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
            a = np.maximum(0, z)  # ReLU Activation
            self.z_values.append(z)
            self.activations.append(a)

        # Output layer (Softmax)
        z_out = np.dot(self.activations[-1], self.weights[-1]) + self.biases[-1]
        a_out = softmax(z_out)
        self.z_values.append(z_out)
        self.activations.append(a_out)

        return a_out

    def backward(self, X, y, learning_rate):
        """
        Backpropagation for multi-layer MLP.
        """
        m = X.shape[0]
        dz = self.activations[-1] - y  # Output layer gradient (Softmax + BCE)

        # Backpropagate through layers
        for i in range(self.num_layers - 1, 0, -1):  # Hidden layers
            dW = np.dot(self.activations[i].T, dz) / m
            db = np.sum(dz, axis=0, keepdims=True) / m
            dz = np.dot(dz, self.weights[i].T) * (self.z_values[i - 1] > 0)  # ReLU derivative

            # Update weights
            self.weights[i] -= learning_rate * dW
            self.biases[i] -= learning_rate * db

        # First layer update
        dW = np.dot(X.T, dz) / m
        db = np.sum(dz, axis=0, keepdims=True) / m
        self.weights[0] -= learning_rate * dW
        self.biases[0] -= learning_rate * db

    def train(self, X, y, learning_rate=0.01, epochs=1000):
        """
        Train the MLP model.
        """
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = binary_cross_entropy(y, y_pred)
            self.backward(X, y, learning_rate)

            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        """
        Predict class labels (0 or 1).
        """
        y_pred = self.forward(X)
        return np.argmax(y_pred, axis=1)  # Return the class with the highest probability

# Generate Synthetic Binary Data
np.random.seed(10121091)

# Generate 2D features (simple binary classification)
X_data = np.random.randn(1000, 2)
y_data = (X_data[:, 0] + X_data[:, 1] > 0).astype(int)  # Class 1 if sum > 0, else Class 0

# Convert labels to one-hot encoding for Softmax
y_data_one_hot = np.zeros((y_data.size, 2))
y_data_one_hot[np.arange(y_data.size), y_data] = 1

# Split into training and testing sets (80% train, 20% test)
split_ratio = 0.7
split_index = int(len(X_data) * split_ratio)

X_train, X_test = X_data[:split_index], X_data[split_index:]
y_train, y_test = y_data_one_hot[:split_index], y_data_one_hot[split_index:]

# Train the Custom MLP

# Define architecture: [input_size, hidden_layer1, hidden_layer2, ..., output_size]
layer_sizes = [2, 20, 10, 5, 2]  # Custom hidden layers

mlp = CustomBinaryMLP(layer_sizes)
mlp.train(X_train, y_train, learning_rate=0.01, epochs=2000)

# Evaluate the Model
y_pred = mlp.predict(X_test)
y_true = np.argmax(y_test, axis=1)
accuracy = np.mean(y_pred == y_true)

print(f"Test Accuracy: {accuracy:.4f}")


Epoch 0, Loss: 1.5042
Epoch 100, Loss: 0.2318
Epoch 200, Loss: 0.1410
Epoch 300, Loss: 0.1035
Epoch 400, Loss: 0.0824
Epoch 500, Loss: 0.0687
Epoch 600, Loss: 0.0591
Epoch 700, Loss: 0.0521
Epoch 800, Loss: 0.0467
Epoch 900, Loss: 0.0425
Epoch 1000, Loss: 0.0391
Epoch 1100, Loss: 0.0363
Epoch 1200, Loss: 0.0338
Epoch 1300, Loss: 0.0318
Epoch 1400, Loss: 0.0300
Epoch 1500, Loss: 0.0284
Epoch 1600, Loss: 0.0270
Epoch 1700, Loss: 0.0258
Epoch 1800, Loss: 0.0247
Epoch 1900, Loss: 0.0237
Test Accuracy: 1.0000


## **3. Multiclassification Using NumPy**

In [ ]:
import numpy as np

# Input Parameters
N = 1000        # Number of data samples
num_classes = 3 # Number of target classes

# Activation Functions
def softmax(z):
    """Softmax activation function applied row-wise."""
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # for numerical stability
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# Categorical Cross-Entropy Loss
def categorical_cross_entropy(y_true, y_pred):
    """Computes Categorical Cross-Entropy loss.

    y_true is one-hot encoded.
    """
    m = y_true.shape[0]
    loss = -np.mean(np.log(y_pred[np.arange(m), np.argmax(y_true, axis=1)] + 1e-9))
    return loss

def generate_labels(X_data, num_classes):
    """
    Generate classification labels for any number of classes using a general rule.
    """
    N = X_data.shape[0]
    y_data = np.zeros(N, dtype=int)  # Default all to class 0

    if num_classes == 1:
        return y_data  # All belong to class 0

    elif num_classes == 2:
        return (X_data[:, 0] > 0).astype(int)  # Right (1) vs Left (0)

    elif num_classes == 3:
        y_data[(X_data[:, 0] > 0) & (X_data[:, 1] > 0)] = 0  # First quadrant
        y_data[(X_data[:, 0] <= 0) & (X_data[:, 1] > 0)] = 1  # Second quadrant
        y_data[X_data[:, 1] <= 0] = 2  # Bottom half

    elif num_classes == 4:
        y_data[(X_data[:, 0] > 0) & (X_data[:, 1] > 0)] = 0  # First quadrant
        y_data[(X_data[:, 0] <= 0) & (X_data[:, 1] > 0)] = 1  # Second quadrant
        y_data[(X_data[:, 0] <= 0) & (X_data[:, 1] <= 0)] = 2  # Third quadrant
        y_data[(X_data[:, 0] > 0) & (X_data[:, 1] <= 0)] = 3  # Fourth quadrant

    elif num_classes == 5:
        center_mask = (np.abs(X_data[:, 0]) < 0.5) & (np.abs(X_data[:, 1]) < 0.5)
        y_data[center_mask] = 4  # Center region
        y_data[(X_data[:, 0] > 0) & (X_data[:, 1] > 0)] = 0  # First quadrant
        y_data[(X_data[:, 0] <= 0) & (X_data[:, 1] > 0)] = 1  # Second quadrant
        y_data[(X_data[:, 0] <= 0) & (X_data[:, 1] <= 0)] = 2  # Third quadrant
        y_data[(X_data[:, 0] > 0) & (X_data[:, 1] <= 0)] = 3  # Fourth quadrant

    else:
        # General rule for any number of classes > 5
        angles = np.arctan2(X_data[:, 1], X_data[:, 0])
        angles[angles < 0] += 2 * np.pi
        sector_size = 2 * np.pi / num_classes
        y_data = (angles // sector_size).astype(int)
    return y_data

# Multi-Class MLP with Custom Hidden Layers
class MultiClassMLP:
    def __init__(self, layer_sizes):
        """
        Initializes a multi-layer perceptron for multi-class classification.
        """
        self.num_layers = len(layer_sizes) - 1
        self.weights = []
        self.biases = []

        for i in range(self.num_layers):
            self.weights.append(np.random.randn(layer_sizes[i], layer_sizes[i+1]) * np.sqrt(2 / layer_sizes[i]))
            self.biases.append(np.zeros((1, layer_sizes[i+1])))

    def forward(self, X):
        """Forward propagation through the network."""
        self.activations = [X]
        self.z_values = []

        for i in range(self.num_layers - 1):
            z = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
            self.z_values.append(z)
            a = np.maximum(0, z)  # ReLU activation
            self.activations.append(a)

        z_out = np.dot(self.activations[-1], self.weights[-1]) + self.biases[-1]
        self.z_values.append(z_out)
        a_out = softmax(z_out)
        self.activations.append(a_out)
        return a_out

    def backward(self, X, y, learning_rate):
        """Backpropagation to compute gradients and update weights."""
        m = X.shape[0]
        delta = self.activations[-1] - y

        for i in range(self.num_layers - 1, 0, -1):
            dW = np.dot(self.activations[i].T, delta) / m
            db = np.sum(delta, axis=0, keepdims=True) / m
            self.weights[i] -= learning_rate * dW
            self.biases[i]  -= learning_rate * db
            delta = np.dot(delta, self.weights[i].T)
            delta = delta * (self.z_values[i-1] > 0)

        dW = np.dot(X.T, delta) / m
        db = np.sum(delta, axis=0, keepdims=True) / m
        self.weights[0] -= learning_rate * dW
        self.biases[0]  -= learning_rate * db

    def train(self, X, y, learning_rate=0.01, epochs=1000):
        """Train the MLP model."""
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = categorical_cross_entropy(y, y_pred)
            self.backward(X, y, learning_rate)
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        """Predict class labels."""
        y_pred = self.forward(X)
        return np.argmax(y_pred, axis=1)

# Generate Synthetic Multi-Class Data
np.random.seed(10121091)
X_data = np.random.randn(N, 2)

y_data = generate_labels(X_data, num_classes)

# One-hot encode labels
y_data_one_hot = np.zeros((N, num_classes))
y_data_one_hot[np.arange(N), y_data] = 1

# Split Data into Training and Testing Sets
split_ratio = 0.7
split_index = int(N * split_ratio)
X_train, X_test = X_data[:split_index], X_data[split_index:]
y_train, y_test = y_data_one_hot[:split_index], y_data_one_hot[split_index:]

# Train the MLP
layer_sizes = [2, 20, 10, num_classes]
mlp = MultiClassMLP(layer_sizes)
mlp.train(X_train, y_train, learning_rate=0.01, epochs=1000)

# Evaluate the Model
y_pred = mlp.predict(X_test)
accuracy = np.mean(y_pred == np.argmax(y_test, axis=1))
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 0, Loss: 1.5228
Epoch 100, Loss: 0.6609
Epoch 200, Loss: 0.4051
Epoch 300, Loss: 0.2831
Epoch 400, Loss: 0.2183
Epoch 500, Loss: 0.1800
Epoch 600, Loss: 0.1547
Epoch 700, Loss: 0.1366
Epoch 800, Loss: 0.1229
Epoch 900, Loss: 0.1122
Test Accuracy: 0.9867


## **4. Multiclassification Using PyTorch**

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Input Parameters
N = 1000           # Number of data samples
num_classes = 8    # Number of target classes

# Data Generation
np.random.seed(10121091)
X_data = np.random.randn(N, 2)

def generate_labels(X_data, num_classes):
    """
    Generate classification labels for any number of classes using a general rule.
    """
    N = X_data.shape[0]
    y_data = np.zeros(N, dtype=int)  # Default all to class 0

    if num_classes == 1:
        return y_data

    elif num_classes == 2:
        return (X_data[:, 0] > 0).astype(int)

    elif num_classes == 3:
        y_data[(X_data[:, 0] > 0) & (X_data[:, 1] > 0)] = 0
        y_data[(X_data[:, 0] <= 0) & (X_data[:, 1] > 0)] = 1
        y_data[X_data[:, 1] <= 0] = 2

    elif num_classes == 4:
        y_data[(X_data[:, 0] > 0) & (X_data[:, 1] > 0)] = 0
        y_data[(X_data[:, 0] <= 0) & (X_data[:, 1] > 0)] = 1
        y_data[(X_data[:, 0] <= 0) & (X_data[:, 1] <= 0)] = 2
        y_data[(X_data[:, 0] > 0) & (X_data[:, 1] <= 0)] = 3

    elif num_classes == 5:
        center_mask = (np.abs(X_data[:, 0]) < 0.5) & (np.abs(X_data[:, 1]) < 0.5)
        y_data[center_mask] = 4
        y_data[(X_data[:, 0] > 0) & (X_data[:, 1] > 0)] = 0
        y_data[(X_data[:, 0] <= 0) & (X_data[:, 1] > 0)] = 1
        y_data[(X_data[:, 0] <= 0) & (X_data[:, 1] <= 0)] = 2
        y_data[(X_data[:, 0] > 0) & (X_data[:, 1] <= 0)] = 3

    else:
        angles = np.arctan2(X_data[:, 1], X_data[:, 0])
        angles[angles < 0] += 2 * np.pi

        sector_size = 2 * np.pi / num_classes
        y_data = (angles // sector_size).astype(int)
    return y_data

y_data = generate_labels(X_data, num_classes)

# Split data (70% train, 30% test)
split_index = int(0.7 * N)
X_train = X_data[:split_index]
X_test  = X_data[split_index:]
y_train = y_data[:split_index]
y_test  = y_data[split_index:]

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor  = torch.tensor(y_test, dtype=torch.long)

# Define the MLP Model
class MLP(nn.Module):
    def __init__(self, layer_sizes):
        """
        layer_sizes: List of layer sizes, e.g., [2, 20, 10, num_classes]
        """
        super(MLP, self).__init__()
        layers = []
        for i in range(len(layer_sizes) - 1):
            layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            if i < len(layer_sizes) - 2:
                layers.append(nn.ReLU())
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

layer_sizes = [2, 20, 10, num_classes]
model = MLP(layer_sizes)

# Apply He initialization
for m in model.modules():
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)

# Training
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
num_epochs = 1000

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Evaluation
with torch.no_grad():
    outputs_test = model(X_test_tensor)
    predicted = torch.argmax(outputs_test, dim=1)
    accuracy = (predicted == y_test_tensor).float().mean().item()
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 0, Loss: 2.5472
Epoch 100, Loss: 0.1419
Epoch 200, Loss: 0.0571
Epoch 300, Loss: 0.0324
Epoch 400, Loss: 0.0185
Epoch 500, Loss: 0.0116
Epoch 600, Loss: 0.0085
Epoch 700, Loss: 0.0067
Epoch 800, Loss: 0.0056
Epoch 900, Loss: 0.0047
Test Accuracy: 0.9767


## **5. Multiclassification Using SKLearn**

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Input Parameters
N = 1000           # Number of data samples
num_classes = 5    # Number of target classes

# Data Generation
np.random.seed(10121091)
X_data = np.random.randn(N, 2)

def generate_labels(X_data, num_classes):
    """
    Generate classification labels for any number of classes using a general rule.
    """
    N = X_data.shape[0]
    y_data = np.zeros(N, dtype=int)  # Default all to class 0

    if num_classes == 1:
        return y_data

    elif num_classes == 2:
        return (X_data[:, 0] > 0).astype(int)

    elif num_classes == 3:
        y_data[(X_data[:, 0] > 0) & (X_data[:, 1] > 0)] = 0
        y_data[(X_data[:, 0] <= 0) & (X_data[:, 1] > 0)] = 1
        y_data[X_data[:, 1] <= 0] = 2

    elif num_classes == 4:
        y_data[(X_data[:, 0] > 0) & (X_data[:, 1] > 0)] = 0
        y_data[(X_data[:, 0] <= 0) & (X_data[:, 1] > 0)] = 1
        y_data[(X_data[:, 0] <= 0) & (X_data[:, 1] <= 0)] = 2
        y_data[(X_data[:, 0] > 0) & (X_data[:, 1] <= 0)] = 3

    elif num_classes == 5:
        center_mask = (np.abs(X_data[:, 0]) < 0.5) & (np.abs(X_data[:, 1]) < 0.5)
        y_data[center_mask] = 4
        y_data[(X_data[:, 0] > 0) & (X_data[:, 1] > 0)] = 0
        y_data[(X_data[:, 0] <= 0) & (X_data[:, 1] > 0)] = 1
        y_data[(X_data[:, 0] <= 0) & (X_data[:, 1] <= 0)] = 2
        y_data[(X_data[:, 0] > 0) & (X_data[:, 1] <= 0)] = 3

    else:
        angles = np.arctan2(X_data[:, 1], X_data[:, 0])
        angles[angles < 0] += 2 * np.pi

        sector_size = 2 * np.pi / num_classes
        y_data = (angles // sector_size).astype(int)
    return y_data

y_data = generate_labels(X_data, num_classes)

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

# Train the MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(20, 10), activation='relu', solver='adam',
                    max_iter=1000, random_state=42)
clf.fit(X_train, y_train)

# Evaluation
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.9900


## **6. Multiclass MLP Reviews Using NumPy**

In [ ]:
import numpy as np
import random

# Seed for reproducibility
random.seed(10121091)
np.random.seed(10121091)

# Sample words for generating synthetic reviews
positive_words = ['excellent', 'great', 'lovely', 'fantastic', 'wonderful']
negative_words = ['terrible', 'poor', 'disappointing', 'bad', 'awful']
neutral_words = ['okay', 'decent', 'average', 'fair', 'satisfactory']
service_words = ['service', 'staff', 'waiter', 'manager', 'support']
price_words = ['price', 'cost', 'charge', 'fee', 'expense']
quality_words = ['quality', 'taste', 'flavor', 'preparation', 'ingredient']
ambiance_words = ['ambiance', 'environment', 'atmosphere', 'setting', 'surroundings']

# Generate synthetic reviews
reviews = []
labels = []
N = 1000  # Number of reviews
for _ in range(N):
    review = []
    label = set()

    # Randomly choose sentiment
    sentiment = random.choice(['positive', 'negative', 'neutral'])
    if sentiment == 'positive':
        review.append(random.choice(positive_words))
        label.add("Positive")
    elif sentiment == 'negative':
        review.append(random.choice(negative_words))
        label.add("Negative")
    else:
        review.append(random.choice(neutral_words))
        label.add("Neutral")

    # Randomly add some topics
    if random.random() < 0.5:
        review.append(random.choice(service_words))
        label.add("Service")
    if random.random() < 0.5:
        review.append(random.choice(price_words))
        label.add("Price")
    if random.random() < 0.5:
        review.append(random.choice(quality_words))
        label.add("Quality")
    if random.random() < 0.5:
        review.append(random.choice(ambiance_words))
        label.add("Ambiance")

    reviews.append(" ".join(review))
    labels.append(label)

# Create a vocabulary and label mapping
vocab = set()
for review in reviews:
    for word in review.split():
        vocab.add(word)
vocab = list(vocab)

label_set = set()
for label in labels:
    label_set.update(label)
label_set = list(label_set)

# Convert reviews to numerical features (bag of words)
X = np.zeros((len(reviews), len(vocab)))
for i, review in enumerate(reviews):
    for word in review.split():
        if word in vocab:
            X[i, vocab.index(word)] += 1

# Convert labels to binary matrix
y = np.zeros((len(labels), len(label_set)))
for i, label in enumerate(labels):
    for l in label:
        y[i, label_set.index(l)] = 1

# Split dataset into training and testing sets
def train_test_split(X, y, test_size=0.3, random_state=None):
    np.random.seed(random_state)
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    split = int(X.shape[0] * (1 - test_size))
    X_train, X_test = X[indices[:split]], X[indices[split:]]
    y_train, y_test = y[indices[:split]], y[indices[split:]]
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Activation functions
def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return (z > 0).astype(float)

def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# Cross-entropy loss function
def cross_entropy_loss(y_true, y_pred):
    m = y_true.shape[0]
    log_likelihood = -np.log(y_pred[range(m), np.argmax(y_true, axis=1)])
    loss = np.sum(log_likelihood) / m
    return loss

# Multi-layer perceptron (MLP) class with customizable hidden layers
class MLP:
    def __init__(self, input_size, hidden_layers, output_size):
        self.hidden_layers = hidden_layers
        self.weights = []
        self.biases = []

        # Initialize weights and biases for each layer
        layer_sizes = [input_size] + hidden_layers + [output_size]
        for i in range(len(layer_sizes) - 1):
            self.weights.append(np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * 0.01)
            self.biases.append(np.zeros((1, layer_sizes[i + 1])))

    def forward(self, X):
        self.activations = [X]
        self.z_values = []

        # Forward pass through each layer
        for i in range(len(self.weights)):
            z = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
            self.z_values.append(z)
            if i < len(self.weights) - 1:
                a = relu(z)  # ReLU for hidden layers
            else:
                a = softmax(z)  # Softmax for output layer
            self.activations.append(a)

        return self.activations[-1]

    def backward(self, X, y, learning_rate):
        m = X.shape[0]
        gradients = []

        # Output layer error
        dz = self.activations[-1] - y
        gradients.append(dz)

        # Backpropagate through hidden layers
        for i in range(len(self.weights) - 1, 0, -1):
            dz = np.dot(gradients[-1], self.weights[i].T) * relu_derivative(self.z_values[i - 1])
            gradients.append(dz)

        gradients.reverse()

        # Update weights and biases
        for i in range(len(self.weights)):
            dW = np.dot(self.activations[i].T, gradients[i]) / m
            db = np.sum(gradients[i], axis=0, keepdims=True) / m
            self.weights[i] -= learning_rate * dW
            self.biases[i] -= learning_rate * db

    def train(self, X, y, learning_rate=0.01, epochs=1000):
        for epoch in range(epochs):
            # Forward pass
            y_pred = self.forward(X)

            # Compute loss
            loss = cross_entropy_loss(y, y_pred)

            # Backward pass
            self.backward(X, y, learning_rate)

            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss}")

    def predict(self, X):
        y_pred = self.forward(X)
        return np.argmax(y_pred, axis=1)

# Define the architecture of the neural network
input_size = X_train.shape[1]
hidden_layers = [20, 20]  # Two hidden layers with 20 neurons each
output_size = y_train.shape[1]

# Initialize and train the MLP
mlp = MLP(input_size, hidden_layers, output_size)
mlp.train(X_train, y_train, learning_rate=0.01, epochs=1000)

# Evaluate the model
y_pred = mlp.predict(X_test)
y_true = np.argmax(y_test, axis=1)
accuracy = np.mean(y_pred == y_true)
print(f"Accuracy: {accuracy}")


Epoch 0, Loss: 1.9459077111545002
Epoch 100, Loss: 1.9361894204703056
Epoch 200, Loss: 1.9325955156493018
Epoch 300, Loss: 1.9331254184296671
Epoch 400, Loss: 1.9368903944342089
Epoch 500, Loss: 1.945029151312743
Epoch 600, Loss: 1.9654954079151006
Epoch 700, Loss: 2.029323670911669
Epoch 800, Loss: inf


<ipython-input-9-2cda16ab238b>:105: RuntimeWarning: divide by zero encountered in log
  log_likelihood = -np.log(y_pred[range(m), np.argmax(y_true, axis=1)])
<ipython-input-9-2cda16ab238b>:99: RuntimeWarning: invalid value encountered in subtract
  exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))


Epoch 900, Loss: nan
Accuracy: 0.5433333333333333


## **7. Multiclass MLP Reviews Using SKLearn**

In [ ]:
import random
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

# Seed for reproducibility
random.seed(10121091)

# Sample words for generating synthetic reviews
positive_words = ['excellent', 'great', 'lovely', 'fantastic', 'wonderful']
negative_words = ['terrible', 'poor', 'disappointing', 'bad', 'awful']
neutral_words = ['okay', 'decent', 'average', 'fair', 'satisfactory']
service_words = ['service', 'staff', 'waiter', 'manager', 'support']
price_words = ['price', 'cost', 'charge', 'fee', 'expense']
quality_words = ['quality', 'taste', 'flavor', 'preparation', 'ingredient']
ambiance_words = ['ambiance', 'environment', 'atmosphere', 'setting', 'surroundings']

# Generate synthetic reviews
reviews = []
labels = []
N = 1000
for _ in range(N):  # Generate 1000 reviews
    review = []
    label = set()

    # Randomly choose sentiment
    sentiment = random.choice(['positive', 'negative', 'neutral'])
    if sentiment == 'positive':
        review.append(random.choice(positive_words))
        label.add("Positive")
    elif sentiment == 'negative':
        review.append(random.choice(negative_words))
        label.add("Negative")
    else:
        review.append(random.choice(neutral_words))
        label.add("Neutral")

    # Randomly add some topics
    if random.random() < 0.5:
        review.append(random.choice(service_words))
        label.add("Service")
    if random.random() < 0.5:
        review.append(random.choice(price_words))
        label.add("Price")
    if random.random() < 0.5:
        review.append(random.choice(quality_words))
        label.add("Quality")
    if random.random() < 0.5:
        review.append(random.choice(ambiance_words))
        label.add("Ambiance")

    reviews.append(" ".join(review))
    labels.append(label)

# Preprocessing
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(reviews).toarray()

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(labels)

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Model
model = OneVsRestClassifier(SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3, verbose=1000))
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


-- Epoch 1
Norm: 43.73, NNZs: 35, Bias: -4.272408, T: 700, Avg. loss: 0.367791
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 35.31, NNZs: 35, Bias: -3.687017, T: 1400, Avg. loss: 0.013813
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 32.69, NNZs: 35, Bias: -3.390818, T: 2100, Avg. loss: 0.021517
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 31.87, NNZs: 35, Bias: -3.186692, T: 2800, Avg. loss: 0.025615
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 31.47, NNZs: 35, Bias: -3.129884, T: 3500, Avg. loss: 0.026651
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 31.32, NNZs: 35, Bias: -2.666942, T: 4200, Avg. loss: 0.027233
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 31.18, NNZs: 35, Bias: -3.043807, T: 4900, Avg. loss: 0.027633
Total training time: 0.00 seconds.
Convergence after 7 epochs took 0.00 seconds
-- Epoch 1
Norm: 34.64, NNZs: 35, Bias: -1.995620, T: 700, Avg. loss: 0.143479
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 27.80, NNZs: 35, Bia